In [8]:
import pandas as pd
from pandas import read_csv
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from pandas import read_csv
from datetime import datetime
from sklearn.metrics import mean_squared_error
import math
from sklearn.neighbors import KNeighborsRegressor
import random
import numpy as np
import numpy.matlib
import sklearn.svm
import weatherdata
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression


In [9]:
# loading separate datasets
mensdataset = read_csv('csv/mens_clothing.csv')
womensdataset = read_csv('csv/womens_clothing.csv')
othersdataset = read_csv('csv/other_clothing.csv')

mensdataset = mensdataset[pd.notnull(mensdataset['Sales(In ThousandDollars)'])]
womensdataset = womensdataset[pd.notnull(womensdataset['Sales(In ThousandDollars)'])]
othersdataset = othersdataset[pd.notnull(othersdataset['Sales(In ThousandDollars)'])]

scale = StandardScaler()


In [10]:
# preprocessing data
def preprocess(dataset,isTest, cate):
    dropcolumns = ['ProductCategory','PartyInPower','AdvertisingExpenses (in Thousand Dollars)','Change(in%)']
    macrodataset = read_csv('csv/train_macro.csv')
    weatherdataset = read_csv('csv/weatherdata_final.csv')
    eventdays = read_csv('csv/eventdays.csv')
    result = pd.merge(dataset, weatherdataset, on=['Year','Month'], how='inner')
    result = pd.merge(result, eventdays, on=['Year','Month'], how='left')
    result = result.fillna(0)
    frames = [result, macrodataset]
    traindataset = pd.concat(frames,axis=1)
    traindataset = traindataset.drop(dropcolumns,axis=1)
    traindataset.fillna(0, inplace=True)
    datavalues = traindataset.values
    xvalues = datavalues[:,3:]
    yvalues = datavalues[:,2]
    xvalues = scale.fit_transform(xvalues)
    return train_test_split(xvalues, yvalues, test_size=0.2, random_state=100)
       


In [11]:
# models
def trainModel(x_train, y_train):    
    regr = linear_model.LinearRegression()
    regr.fit(x_train, y_train)
    return regr

def knnModel(x_train,y_train, k):
    neigh = KNeighborsRegressor(n_neighbors=k)
    neigh.fit(x_train, y_train)
    return neigh

def svrModel(x_train,y_train):
    tuned_model = sklearn.svm.SVR(kernel = 'linear',C=1)
    tuned_model.fit(x_train, y_train)
    return tuned_model

def ridgeMode(x_train,y_train):
    clf = Ridge(alpha=1.0)
    clf.fit(x_train, y_train) 
    return clf


In [12]:
def drwaplots(meny_test,men_predict,womeny_test, women_predict,othersy_test,other_predict ):
    plt.figure()
    plt.subplot(311)
    plt.plot(meny_test,  color='black')
    plt.plot(men_predict, color='blue')
    plt.title('mens prediction')
    
    plt.subplot(312)
    plt.plot(womeny_test,  color='black')
    plt.plot(women_predict, color='blue')
    plt.title('womens prediction')
    
    plt.subplot(313)
    plt.plot(othersy_test,  color='black')
    plt.plot(other_predict, color='blue')
    plt.title('others prediction')
    plt.show()

In [13]:
isTest = False
menx_train, menx_test, meny_train, meny_test =  preprocess(mensdataset, isTest, 'men')
womenx_train, womenx_test, womeny_train, womeny_test =  preprocess(womensdataset,isTest, 'women' )
othersx_train, othersx_test, othersy_train, othersy_test =  preprocess(othersdataset, isTest, 'others')

menx_train.shape,menx_test.shape,meny_train.shape,meny_test.shape

((57, 24), (15, 24), (57,), (15,))

In [23]:
# using knn

param_grid = {'n_neighbors': [3,4,5,6,7,8,9], 'weights': ['uniform','distance'], 'algorithm' : ['brute', 'kd_tree', 'ball_tree', 'auto'],
  'p' :[1,2,3,4,5,6], 'leaf_size' : [20,30,40] }

neigh = KNeighborsRegressor()
menregr = GridSearchCV(neigh, param_grid)
menregr.fit(menx_train,meny_train)
print(menregr.best_params_)

neigh = KNeighborsRegressor()
womenregr = GridSearchCV(neigh, param_grid)
womenregr.fit(womenx_train,womeny_train)
print(womenregr.best_params_)


neigh = KNeighborsRegressor()
othersregr = GridSearchCV(neigh, param_grid)
othersregr.fit(othersx_train,othersy_train)
print(othersregr.best_params_)


men_predict = menregr.predict(menx_test)
women_predict = womenregr.predict(womenx_test)
other_predict = othersregr.predict(othersx_test)

men_rmse = math.sqrt(mean_squared_error(men_predict, meny_test))
women_rmse = math.sqrt(mean_squared_error(women_predict, womeny_test))
other_rmse = math.sqrt(mean_squared_error(other_predict, othersy_test))

print('men_rmse: %.3f' % men_rmse, "women_rmse: %.3f" % women_rmse, 'others rmse: %.3f' % other_rmse)
    

{'weights': 'distance', 'p': 6, 'leaf_size': 20, 'algorithm': 'brute', 'n_neighbors': 3}
{'weights': 'distance', 'p': 3, 'leaf_size': 20, 'algorithm': 'brute', 'n_neighbors': 3}
{'weights': 'distance', 'p': 4, 'leaf_size': 20, 'algorithm': 'brute', 'n_neighbors': 3}
men_rmse: 83.576 women_rmse: 470.570 others rmse: 125.210


In [27]:
testdata = pd.read_csv('csv/test_data.csv')
testMacroData = pd.read_csv('csv/test_macro.csv')
weather = pd.read_csv("csv/weatherdata_final.csv", sep=",")
result = pd.merge(testdata, weather, on=['Year','Month'], how='inner')
eventdays = read_csv('csv/eventdays.csv')
result = pd.merge(result, eventdays, on=['Year','Month'], how='left')
result = result.fillna(0)
del result['Sales(In ThousandDollars)']
#result

In [28]:
# repeating macro business data for 3 categories
def preprocessTestMacroData(testMacroData):
    dropcolumns = ['PartyInPower','AdvertisingExpenses (in Thousand Dollars)','Change(in%)', 'Year','Month','Year-Month']
    testMacroData = testMacroData.drop(dropcolumns,axis=1)

    testMacroData.fillna(0)
    finalData = np.zeros((3*testMacroData.values.shape[0],testMacroData.values.shape[1]),dtype = object)
    for i,row in enumerate(testMacroData.values):    
        x =  np.array(row)
        x = np.matlib.repmat(x,3,1)
        finalData[i*3:3*i+3] = x
    changed = pd.DataFrame(finalData, columns = testMacroData.columns)
    return changed

In [29]:
test_macro = preprocessTestMacroData(testMacroData)
frames = [result, test_macro]
testdataset = pd.concat(frames,axis=1)
#testdataset

In [32]:
mensdata = []
womensdata = []
othersdata = []
data = testdataset.values
#ata = scale.transform(data)
for row in data:
    if row[2] == 'WomenClothing':
        womensdata.append(row[3:])
    elif row[2] == 'MenClothing':
        mensdata.append(row[3:])
    else:
        othersdata.append(row[3:])
mensdata = np.array(mensdata)
print((mensdata).shape)


mensdata = scale.transform(mensdata)
womensdata = scale.transform(womensdata)
othersdata = scale.transform(othersdata)

#mensdata = pca.transform(mensdata)
#womensdata = pca.transform(womensdata)
#othersdata = pca.transform(othersdata)

men_predict = menregr.predict(mensdata)
women_predict = womenregr.predict(womensdata)
other_predict = othersregr.predict(othersdata)
predict = []
#rint(testdataset.values.shape)
for i in range(testdataset.values.shape[0]//3):
    predict.append(women_predict[i])
    predict.append(men_predict[i])
    predict.append(other_predict[i])

print(predict)
index = [i for i in range(1,37)]
out = [ [i+1, pred] for i,pred in enumerate(predict)]
output = pd.DataFrame(out,columns = ['Year','Sales(In ThousandDollars)'])
#originalData = testdataset.values[:,:3]
#testdata['Sales(In ThousandDollars)'] = predict
#testdataset
output.to_csv('csv/test_output.csv', sep=',', index=False)


(12, 24)
[4075.703223190013, 708.4250306187542, 1228.9836976558774, 4054.193250055218, 719.3369488862527, 1227.77665124128, 4124.606454784451, 718.5170191266385, 1204.9773546365311, 3919.0123867888046, 731.3405469054802, 1228.2056363163654, 4131.116265986823, 735.0963702589945, 1190.0818864587675, 4172.778879991532, 731.4202427480325, 1236.211024888952, 4125.806384114843, 718.1563231050108, 1249.3947076802672, 3926.508041910963, 718.3491554324476, 1197.7488751405635, 4110.545088376099, 724.8673164011043, 1193.2469799159758, 4074.2553713138736, 729.4859891850077, 1260.9246347411347, 4076.9733009014803, 698.5411271668161, 1263.5000753309123, 5031.8708674050185, 1078.8698496025538, 1598.0093229878996]


/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
